# Labelme Assignment

### Purpose:
Demarcate satellite images that contain fire programmatically 

### Steps:

* Read the fire perimeters data. This shud contain:
    * Fire Start date
    * Centroid (X & Y)
    * Fire Perimeter Coordinates
* Download Landsat Path Row shapefile
* Download Landsat scenes as CSV
* Identify satellite image corresponding which intersects with each fire centroid and fire start date
    * More than 1 image per location- time is possible
        * In this case, select onw with least cloud cover
    * No images is also possible
        * Do something - Not solved now
* Create RGB composite
* Subset **200 * 200** pixels centered around fire centroid (Each px is **30 m**, so each scene now is **6 sq.km**)
* Find polygon coordinates in terms of pixels
* Create json in the format required by labelme
    * The polygon created in previous step becomes the annotated polygon
    * In case a different label (say non-fire burnt area is required, use images with dates after fire is contained)

## Install following libraries

* geopandas
* descartes
* rasterio
* folium 
* cartopy (conda install) (conda install -c conda-forge cartopy)

In [2]:
!pip install geopandas
!pip install descartes
!pip install folium
!pip install rasterio

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Read Fire Perimeter data

In [4]:
import pandas as pd
import geopandas as gpd
import folium
import os

In [5]:
label_df = pd.read_csv("Fire_Prediction/study_area.csv", parse_dates=["ALARM_DATE", "CONT_DATE"])
label_df.tail()

,X,Y,ALARM_DATE,CONT_DATE,CAUSE,minx,miny,maxx,maxy,coords
934,-123.129416,41.957410,2017-08-10,2018-01-10,1.0,-123.260962,41.873994,-123.011714,42.025643,"[(-123.16235357591204, 41.97429019563638), (-1..."
935,-123.289808,41.970587,2017-08-11,2017-11-28,1.0,-123.308693,41.963763,-123.270366,41.978069,"[(-123.30488330805754, 41.96616010635459), (-1..."
936,-120.208866,38.824305,2017-08-11,2017-08-14,14.0,-120.210236,38.822774,-120.207569,38.825822,"[(-120.20756925443563, 38.823916868756854), (-..."
937,-117.577833,33.432467,2017-06-28,2017-06-30,9.0,-117.588969,33.421134,-117.567590,33.443179,"[(-117.56897743867438, 33.439482147245116), (-..."
938,-122.328469,41.433370,2016-05-19,2016-05-20,9.0,-122.331842,41.429042,-122.324932,41.437794,"[(-122.33068299055088, 41.43772946193435), (-1..."


## Download Landsat Path Row shapefile 

Run the following command once

!wget https://landsat.usgs.gov/sites/default/files/documents/WRS2_descending.zip
!unzip WRS2_descending.zip

## Download Landsat Path Row shapefile (WRS)

In [6]:
wrs = gpd.read_file('WRS2_descending.shp')

wrs.head()

,AREA,PERIMETER,PR_,PR_ID,RINGS_OK,RINGS_NOK,PATH,ROW,MODE,SEQUENCE,WRSPR,PR,ACQDayL7,ACQDayL8,geometry
0,15.74326,26.98611,1.0,1.0,1,0,13,1,D,2233,013001,013001,1,9,"POLYGON ((-10.80341356392465 80.9888, -8.97406..."
1,14.55366,25.84254,2.0,2.0,1,0,13,2,D,2234,013002,013002,1,9,POLYGON ((-29.24250366707619 80.18681161921363...
2,13.37247,24.20303,3.0,3.0,1,0,13,3,D,2235,013003,013003,1,9,POLYGON ((-24.04205646041896 79.12261247629547...
3,12.26691,22.40265,4.0,4.0,1,0,13,4,D,2236,013004,013004,1,9,POLYGON ((-36.66813132081753 77.46094098591608...
4,11.26511,20.64284,5.0,5.0,1,0,13,5,D,2237,013005,013005,1,9,POLYGON ((-44.11209517917457 76.93655561966702...


## Download all available landsat scenes from AWS S3 as CSV

In [7]:
s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', \
                        compression='gzip', parse_dates=["acquisitionDate"])


In [8]:
s3_scenes.tail()

,productId,entityId,acquisitionDate,cloudCover,processingLevel,path,row,min_lat,min_lon,max_lat,max_lon,download_url
1464916,LC08_L1TP_221079_20190324_20190324_01_RT,LC82210792019083LGN00,2019-03-24 13:17:29.216514,15.75,L1TP,221,79,-28.48774,-51.80815,-26.37810,-49.44518,https://s3-us-west-2.amazonaws.com/landsat-pds...
1464917,LC08_L1TP_221080_20190324_20190324_01_RT,LC82210802019083LGN00,2019-03-24 13:17:53.154148,3.20,L1TP,221,80,-29.91984,-52.19239,-27.81358,-49.79621,https://s3-us-west-2.amazonaws.com/landsat-pds...
1464918,LC08_L1TP_221082_20190324_20190324_01_RT,LC82210822019083LGN00,2019-03-24 13:18:41.054835,0.46,L1TP,221,82,-32.79625,-52.99138,-30.68220,-50.52473,https://s3-us-west-2.amazonaws.com/landsat-pds...
1464919,LC08_L1TP_221083_20190324_20190324_01_RT,LC82210832019083LGN00,2019-03-24 13:19:05.005178,11.44,L1TP,221,83,-34.23394,-53.40531,-32.11502,-50.89793,https://s3-us-west-2.amazonaws.com/landsat-pds...
1464920,LC08_L1TP_221084_20190324_20190324_01_RT,LC82210842019083LGN00,2019-03-24 13:19:28.951285,14.02,L1TP,221,84,-35.67014,-53.82826,-33.54575,-51.28397,https://s3-us-west-2.amazonaws.com/landsat-pds...


In [17]:
from datetime import timedelta  
from shapely.geometry import Point
import numpy as np

In [28]:
from datetime import datetime  

s3_scenes = s3_scenes[(s3_scenes['acquisitionDate'] >= datetime(2015, 1, 1)) & 
          (s3_scenes['acquisitionDate'] < datetime(2018, 1, 1))]

## ^^ Need to only work with scenes in our study time period 
len(s3_scenes)

667006

## Identify satellite image corresponding to each fire centroid

In [150]:
# Empty list to add the images
bulk_list = []

for i, record in label_df.iloc[199:].iterrows():
    #create a shaply point geometry using X and Y coordinate
    pt = Point(record["X"], record["Y"])
    wrs_intersection = wrs[wrs.intersects(pt)]
    paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values

    # Iterate through paths and rows
    for path, row in zip(paths, rows):

        # Filter the Landsat Amazon S3 table for images matching path, row, cloudcover and processing state.
        scenes = s3_scenes[(s3_scenes.path == path) & (s3_scenes.row == row) & 
                           (~s3_scenes.productId.str.contains('_T2')) &
                           (~s3_scenes.productId.str.contains('_RT')) & 
                           (s3_scenes['acquisitionDate'] >= record.CONT_DATE) & 
                           (s3_scenes['acquisitionDate'] <= record.CONT_DATE+timedelta(days=15))]
        
        print('Path:',path, 'Row:', row,  record.ALARM_DATE, ' till ', record.CONT_DATE, "Scenes : ", len(scenes))


        # If any scenes exists, select the one that have the minimum cloudCover.
        if len(scenes):
            scene = scenes.sort_values('cloudCover').iloc[0]
            # Add the selected scene to the bulk download list.
            scene = pd.concat([scene, record.T])
            bulk_list.append(scene)
print('Done')

Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-10-05 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-10-30 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-10-30 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-10-30 00:00:00 Scenes :  1
Path: 46 Row: 32 2015-07-30 00:00:00  till  2015-10-30 00:00:00 Scenes :  0
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-10-30 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-08-02 00:00:00 Scenes :  1
Path: 46 Row: 32 2015-07-30 00:00:00  till  2015-08-02 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-08-06 00:00:00 Scenes :  1
Path: 46 Row: 32 2015-07-30 00:00:00  till  2015-08-06 00:00:00 Scenes :  1
Path: 45 Row: 32 2015-07-30 00:00:00  till  2015-08-08 00:00:00 Scenes :  1
Path: 46 Row: 31 2015-07-30 00:00:00  till  2015-08-08 00:00:00 Scenes :  1
Path: 46 Row: 32 2015-07-30 00:00:00  till  2015-08-08 00:00:00 Scenes :  1
Path: 45 Row

Path: 42 Row: 35 2016-07-01 00:00:00  till  2016-07-04 00:00:00 Scenes :  1
Path: 42 Row: 36 2016-07-01 00:00:00  till  2016-07-04 00:00:00 Scenes :  1
Path: 41 Row: 36 2016-05-29 00:00:00  till  2016-05-29 00:00:00 Scenes :  1
Path: 41 Row: 36 2016-07-03 00:00:00  till  2016-07-03 00:00:00 Scenes :  1
Path: 42 Row: 36 2016-08-24 00:00:00  till  2016-08-24 00:00:00 Scenes :  1
Path: 45 Row: 32 2016-08-03 00:00:00  till  2016-08-03 00:00:00 Scenes :  1
Path: 44 Row: 32 2016-08-03 00:00:00  till  2016-08-03 00:00:00 Scenes :  1
Path: 45 Row: 32 2016-09-13 00:00:00  till  2016-09-13 00:00:00 Scenes :  1
Path: 44 Row: 32 2016-09-13 00:00:00  till  2016-09-13 00:00:00 Scenes :  1
Path: 45 Row: 32 2016-09-16 00:00:00  till  2016-09-16 00:00:00 Scenes :  1
Path: 44 Row: 32 2016-09-16 00:00:00  till  2016-09-16 00:00:00 Scenes :  0
Path: 43 Row: 34 2016-06-27 00:00:00  till  2016-06-30 00:00:00 Scenes :  1
Path: 44 Row: 33 2016-06-27 00:00:00  till  2016-06-30 00:00:00 Scenes :  1
Path: 43 Row

Path: 43 Row: 35 2016-05-22 00:00:00  till  2016-05-25 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-07-09 00:00:00  till  2016-07-10 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-06-17 00:00:00  till  2016-06-21 00:00:00 Scenes :  0
Path: 43 Row: 35 2016-08-14 00:00:00  till  2016-08-14 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-08-22 00:00:00  till  2016-08-22 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-05-17 00:00:00  till  2016-05-17 00:00:00 Scenes :  1
Path: 44 Row: 34 2016-05-17 00:00:00  till  2016-05-17 00:00:00 Scenes :  1
Path: 44 Row: 35 2016-05-17 00:00:00  till  2016-05-17 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-07-22 00:00:00  till  2016-07-24 00:00:00 Scenes :  1
Path: 43 Row: 34 2016-06-07 00:00:00  till  2016-06-08 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-06-07 00:00:00  till  2016-06-08 00:00:00 Scenes :  1
Path: 44 Row: 34 2016-06-07 00:00:00  till  2016-06-08 00:00:00 Scenes :  1
Path: 43 Row: 35 2016-06-03 00:00:00  till  2016-06-04 00:00:00 Scenes :  1
Path: 44 Row

Path: 40 Row: 36 2016-07-21 00:00:00  till  2016-07-23 00:00:00 Scenes :  1
Path: 41 Row: 36 2016-09-03 00:00:00  till  2016-09-04 00:00:00 Scenes :  1
Path: 40 Row: 36 2016-09-03 00:00:00  till  2016-09-04 00:00:00 Scenes :  0
Path: 40 Row: 36 2016-08-07 00:00:00  till  NaT Scenes :  0
Path: 41 Row: 36 2016-08-16 00:00:00  till  2016-08-26 00:00:00 Scenes :  0
Path: 40 Row: 36 2016-08-16 00:00:00  till  2016-08-26 00:00:00 Scenes :  1
Path: 41 Row: 35 2016-10-20 00:00:00  till  2016-10-21 00:00:00 Scenes :  1
Path: 42 Row: 35 2016-10-20 00:00:00  till  2016-10-21 00:00:00 Scenes :  1
Path: 43 Row: 33 2016-10-03 00:00:00  till  2016-11-15 00:00:00 Scenes :  1
Path: 43 Row: 33 2016-06-28 00:00:00  till  2016-11-08 00:00:00 Scenes :  1
Path: 44 Row: 33 2016-06-28 00:00:00  till  2016-11-08 00:00:00 Scenes :  1
Path: 42 Row: 34 2016-08-03 00:00:00  till  2016-08-08 00:00:00 Scenes :  1
Path: 42 Row: 34 2016-06-24 00:00:00  till  2016-07-07 00:00:00 Scenes :  1
Path: 42 Row: 34 2016-09-17 

Path: 42 Row: 36 2017-10-24 00:00:00  till  2017-10-26 00:00:00 Scenes :  1
Path: 42 Row: 35 2017-06-07 00:00:00  till  2017-06-11 00:00:00 Scenes :  2
Path: 41 Row: 35 2017-07-30 00:00:00  till  2017-08-01 00:00:00 Scenes :  1
Path: 42 Row: 35 2017-07-30 00:00:00  till  2017-08-01 00:00:00 Scenes :  1
Path: 42 Row: 35 2017-07-20 00:00:00  till  2017-07-22 00:00:00 Scenes :  1
Path: 45 Row: 32 2017-10-30 00:00:00  till  2017-10-31 00:00:00 Scenes :  1
Path: 44 Row: 32 2017-10-30 00:00:00  till  2017-10-31 00:00:00 Scenes :  1
Path: 45 Row: 32 2017-06-18 00:00:00  till  2017-06-21 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-06-21 00:00:00  till  2017-06-21 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-08-01 00:00:00  till  2017-08-01 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-07-05 00:00:00  till  2017-07-05 00:00:00 Scenes :  2
Path: 43 Row: 34 2017-09-24 00:00:00  till  2017-09-24 00:00:00 Scenes :  1
Path: 44 Row: 33 2017-09-24 00:00:00  till  2017-09-24 00:00:00 Scenes :  1
Path: 43 Row

...

Path: 44 Row: 33 2017-07-06 00:00:00  till  2017-07-12 00:00:00 Scenes :  4
Path: 45 Row: 33 2017-08-27 00:00:00  till  2017-08-28 00:00:00 Scenes :  3
Path: 44 Row: 34 2017-08-27 00:00:00  till  2017-08-28 00:00:00 Scenes :  3
Path: 44 Row: 33 2017-09-23 00:00:00  till  2017-09-23 00:00:00 Scenes :  1
Path: 45 Row: 33 2017-10-07 00:00:00  till  2017-10-08 00:00:00 Scenes :  1
Path: 45 Row: 33 2017-09-20 00:00:00  till  2017-09-21 00:00:00 Scenes :  2
Path: 44 Row: 33 2017-09-20 00:00:00  till  2017-09-21 00:00:00 Scenes :  1
Path: 45 Row: 33 2017-09-05 00:00:00  till  2017-09-06 00:00:00 Scenes :  3
Path: 44 Row: 33 2017-09-05 00:00:00  till  2017-09-06 00:00:00 Scenes :  2
Path: 45 Row: 33 2017-09-01 00:00:00  till  2017-09-02 00:00:00 Scenes :  3
Path: 45 Row: 33 2017-10-06 00:00:00  till  2017-10-08 00:00:00 Scenes :  1
Path: 44 Row: 33 2017-10-06 00:00:00  till  2017-10-08 00:00:00 Scenes :  2
Path: 45 Row: 33 2017-08-30 00:00:00  till  2017-08-30 00:00:00 Scenes :  3
Path: 44 Row

Path: 43 Row: 32 2017-08-26 00:00:00  till  2017-08-27 00:00:00 Scenes :  0
Path: 44 Row: 32 2017-08-26 00:00:00  till  2017-08-27 00:00:00 Scenes :  3
Path: 43 Row: 32 2017-08-29 00:00:00  till  2017-08-30 00:00:00 Scenes :  3
Path: 44 Row: 32 2017-08-29 00:00:00  till  2017-08-30 00:00:00 Scenes :  3
Path: 43 Row: 32 2017-08-29 00:00:00  till  2017-08-30 00:00:00 Scenes :  3
Path: 43 Row: 32 2017-08-29 00:00:00  till  2017-09-11 00:00:00 Scenes :  3
Path: 44 Row: 32 2017-08-29 00:00:00  till  2017-09-11 00:00:00 Scenes :  1
Path: 43 Row: 32 2017-08-29 00:00:00  till  2017-08-31 00:00:00 Scenes :  3
Path: 44 Row: 32 2017-08-29 00:00:00  till  2017-08-31 00:00:00 Scenes :  3
Path: 43 Row: 32 2017-09-06 00:00:00  till  2017-09-11 00:00:00 Scenes :  3
Path: 44 Row: 32 2017-09-06 00:00:00  till  2017-09-11 00:00:00 Scenes :  1
Path: 40 Row: 36 2017-06-17 00:00:00  till  2017-06-17 00:00:00 Scenes :  1
Path: 40 Row: 36 2017-05-14 00:00:00  till  2017-05-14 00:00:00 Scenes :  1
Path: 40 Row

Path: 45 Row: 32 2017-10-14 00:00:00  till  2017-10-14 00:00:00 Scenes :  1
Path: 46 Row: 32 2017-10-14 00:00:00  till  2017-10-14 00:00:00 Scenes :  3
Path: 45 Row: 32 2017-08-07 00:00:00  till  2017-08-11 00:00:00 Scenes :  3
Path: 46 Row: 32 2017-08-07 00:00:00  till  2017-08-11 00:00:00 Scenes :  1
Path: 45 Row: 32 2017-08-01 00:00:00  till  2017-08-17 00:00:00 Scenes :  3
Path: 46 Row: 32 2017-08-01 00:00:00  till  2017-08-17 00:00:00 Scenes :  3
Path: 45 Row: 31 2017-07-25 00:00:00  till  2017-10-15 00:00:00 Scenes :  1
Path: 46 Row: 31 2017-07-25 00:00:00  till  2017-10-15 00:00:00 Scenes :  2
Path: 45 Row: 32 2017-08-07 00:00:00  till  2017-08-22 00:00:00 Scenes :  3
Path: 46 Row: 32 2017-08-07 00:00:00  till  2017-08-22 00:00:00 Scenes :  3
Path: 41 Row: 35 2017-08-20 00:00:00  till  2018-01-15 00:00:00 Scenes :  0
Path: 42 Row: 34 2017-08-20 00:00:00  till  2018-01-15 00:00:00 Scenes :  0
Path: 42 Row: 35 2017-08-20 00:00:00  till  2018-01-15 00:00:00 Scenes :  0
Path: 41 Row

Path: 41 Row: 36 2017-04-30 00:00:00  till  2017-04-30 00:00:00 Scenes :  1
Path: 40 Row: 36 2017-04-30 00:00:00  till  2017-04-30 00:00:00 Scenes :  1
Path: 40 Row: 36 2017-07-04 00:00:00  till  2017-07-06 00:00:00 Scenes :  4
Path: 40 Row: 36 2017-07-22 00:00:00  till  2017-07-23 00:00:00 Scenes :  1
Path: 41 Row: 36 2017-07-13 00:00:00  till  2017-07-13 00:00:00 Scenes :  1
Path: 41 Row: 36 2017-05-28 00:00:00  till  2017-05-28 00:00:00 Scenes :  2
Path: 41 Row: 36 2017-08-06 00:00:00  till  2017-08-06 00:00:00 Scenes :  1
Path: 41 Row: 36 2017-10-17 00:00:00  till  2017-10-31 00:00:00 Scenes :  1
Path: 41 Row: 36 2017-06-17 00:00:00  till  2017-06-17 00:00:00 Scenes :  1
Path: 40 Row: 36 2017-06-17 00:00:00  till  2017-06-17 00:00:00 Scenes :  1
Path: 44 Row: 31 2017-06-27 00:00:00  till  2017-06-27 00:00:00 Scenes :  1
Path: 44 Row: 31 2017-07-28 00:00:00  till  2017-10-17 00:00:00 Scenes :  0
Path: 45 Row: 31 2017-07-13 00:00:00  till  2017-07-18 00:00:00 Scenes :  1
Path: 44 Row

Path: 42 Row: 34 2017-09-07 00:00:00  till  2017-09-15 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-08-02 00:00:00  till  2017-11-02 00:00:00 Scenes :  1
Path: 42 Row: 34 2017-08-02 00:00:00  till  2017-11-02 00:00:00 Scenes :  1
Path: 45 Row: 31 2017-07-27 00:00:00  till  2017-08-12 00:00:00 Scenes :  3
Path: 44 Row: 31 2017-07-27 00:00:00  till  2017-08-12 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-08-02 00:00:00  till  2017-11-03 00:00:00 Scenes :  1
Path: 42 Row: 34 2017-08-02 00:00:00  till  2017-11-03 00:00:00 Scenes :  1
Path: 41 Row: 36 2017-09-01 00:00:00  till  2017-09-08 00:00:00 Scenes :  2
Path: 43 Row: 31 2017-08-03 00:00:00  till  2017-08-15 00:00:00 Scenes :  1
Path: 44 Row: 31 2017-08-03 00:00:00  till  2017-08-15 00:00:00 Scenes :  1
Path: 43 Row: 34 2017-08-13 00:00:00  till  2017-11-03 00:00:00 Scenes :  1
Path: 42 Row: 34 2017-08-13 00:00:00  till  2017-11-03 00:00:00 Scenes :  1
Path: 45 Row: 33 2017-10-08 00:00:00  till  2017-10-25 00:00:00 Scenes :  1
Path: 45 Row

In [151]:
#creating dataframe
bulk_frame = pd.concat(bulk_list, 1).T
bulk_frame.tail().T

,1001,1002,1003,1004,1005
productId,LC08_L1TP_045033_20171027_20171108_01_T1,LC08_L1TP_046031_20171205_20171222_01_T1,LC08_L1TP_043033_20170826_20170913_01_T1,LC08_L1TP_040037_20170704_20170716_01_T1,LC08_L1TP_045031_20160602_20170223_01_T1
entityId,LC80450332017300LGN00,LC80460312017339LGN00,LC80430332017238LGN00,LC80400372017185LGN00,LC80450312016154LGN01
acquisitionDate,2017-10-27 18:52:09.859558,2017-12-05 18:57:23.520903,2017-08-26 18:39:35.566516,2017-07-04 18:22:20.474080,2016-06-02 18:50:43.984430
cloudCover,36.21,6.96,3.47,16.67,11.49
processingLevel,L1TP,L1TP,L1TP,L1TP,L1TP
path,45,46,43,40,45
row,33,31,33,37,31
min_lat,37.8264,40.6646,37.8186,32.1078,40.6785
min_lon,-124.607,-125.299,-121.46,-118.403,-123.707
max_lat,39.9532,42.8057,39.967,34.2233,42.8158


## Creating Label Me Json

In [106]:
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from rasterio.plot import show
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon
%matplotlib inline

In [114]:
import json
d = json.load(open("Labelme.json", "r"))

In [ ]:
# Import requests and beautiful soup
import requests
from rasterio import features
import copy
import base64
try:
    import io
except ImportError:
    import io as io


# For each row

entity_dir = "Crimson_Red_Sky/Fire"
os.makedirs(entity_dir, exist_ok=True)
os.makedirs(f"{entity_dir}/json", exist_ok=True)


for i, record in bulk_frame.iterrows():
    

    print('EntityId:', record.productId)
    
    url = bulk_frame['download_url'][0].replace("index.html", bulk_frame['productId'][0] + "_B{}.TIF")
    with rasterio.open(url.format(4)) as band4:
        with rasterio.open(url.format(3)) as band3:
            with rasterio.open(url.format(2)) as band2:
                    r = band4.read(1)
                    g = band3.read(1)
                    b = band2.read(1)
        
        # Get Reference system of Landsat image from rasterio
    utm = pyproj.Proj(band4.crs) 
    lonlat = pyproj.Proj(init='epsg:4326')
    lon,lat = record["X"], record["Y"]

    #Project Fire centroid lat lon values into projected coordinates as east, north (UTM - in m)
    east,north = pyproj.transform(lonlat, utm, lon, lat)

    #get pixel values at the centroid corrsponding to projected coordinates
    row, col = band4.index(east, north)

    #Project Fire Perimeter coordinates into UTM coorindates (m)
    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'), # source coordinate system
        pyproj.Proj(init=band4.crs.data["init"])) # destination coordinate system

    g2 = transform(project, Polygon(eval(record.coords)))  # apply projection

    #Get pixel coordinates of fire perimeter

    subset_r = r[row - 250 : row + 250 , col - 250 : col + 250]
    subset_g = g[row - 250 : row + 250 , col - 250 : col + 250]
    subset_b = b[row - 250 : row + 250 , col - 250 : col + 250]
    
    with rasterio.open(f"{entity_dir}/{record.productId}.png", 'w',  driver='PNG',
            count=3,
            dtype=rasterio.uint16, width=500,
            height=500) as out:
        for k, arr in [(1, subset_r), (2, subset_g), (3, subset_b)]:
            out.write_band(k, arr)

    with open(f"{entity_dir}/{record.productId}.png", "rb") as image:
        encoded_string = base64.b64encode(image.read())
        
    if i <= 200 :
        label = "Fire"
    elif i <= 400:
        
        entity_dir = "Crimson_Red_Sky/Non Fire"
        os.makedirs(entity_dir, exist_ok=True)
        os.makedirs(f"{entity_dir}/json", exist_ok=True)
        label = "Non Fire"
    if i > 250:
        entity_dir = "Crimson_Red_Sky/Test"
        os.makedirs(entity_dir, exist_ok=True)

        continue

    poly = [band4.index(coord[0] , coord[1]) for coord in zip(*g2.exterior.xy)]
    
    opti_poly = []
    for coord in poly:
        x = coord[0] - row + 250
        y = coord[1] - col + 250

        if x > 500: 
            x = 500
        if x < 0:
            x = 0
        if y > 500: 
            y = 500
        if y < 0:
            y = 0
        opti_poly.append((x,y))
  
    

    e = copy.deepcopy(d)
    

    e["shapes"][0]['label'] = label
    e["shapes"][0]['points'] = np.take(opti_poly, np.sort(np.random.choice(range(len(opti_poly)), 15, )), axis = 0).tolist()

    e['imageData'] = encoded_string.decode('ascii')
    e['imageWidth'] = 500
    e['imageHeight'] = 500
    e['imagePath'] = f"{entity_dir}/{record.productId}.png"

    json.dump(e, open(f"{entity_dir}/json/{record.productId}.json", "w"))        
        
print('Done')

EntityId: LC08_L1TP_045032_20151006_20170225_01_T1


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/rasterio/__init__.py:225: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  **kwargs)


EntityId: LC08_L1TP_045032_20151107_20170225_01_T1
EntityId: LC08_L1TP_045032_20151107_20170225_01_T1
EntityId: LC08_L1TP_045032_20151107_20170225_01_T1
EntityId: LC08_L1TP_045032_20151107_20170225_01_T1
EntityId: LC08_L1TP_045032_20150803_20170226_01_T1
EntityId: LC08_L1TP_046032_20150810_20170226_01_T1
EntityId: LC08_L1TP_045032_20150819_20180131_01_T1
EntityId: LC08_L1TP_046032_20150810_20170226_01_T1
EntityId: LC08_L1TP_045032_20150819_20180131_01_T1
EntityId: LC08_L1TP_046031_20150810_20170226_01_T1
EntityId: LC08_L1TP_046032_20150810_20170226_01_T1
EntityId: LC08_L1TP_045032_20150819_20180131_01_T1
EntityId: LC08_L1TP_046031_20150810_20170226_01_T1
EntityId: LC08_L1TP_046032_20150810_20170226_01_T1
EntityId: LC08_L1TP_045032_20150819_20180131_01_T1
EntityId: LC08_L1TP_046031_20150810_20170226_01_T1
EntityId: LC08_L1TP_046032_20150810_20170226_01_T1
EntityId: LC08_L1TP_045032_20151107_20170225_01_T1
EntityId: LC08_L1TP_045032_20151022_20170225_01_T1
EntityId: LC08_L1TP_046032_2015

EntityId: LC08_L1TP_043035_20160604_20170223_01_T1
EntityId: LC08_L1TP_043035_20160503_20180201_01_T1
EntityId: LC08_L1TP_042035_20160512_20170223_01_T1
EntityId: LC08_L1TP_043035_20160706_20170221_01_T1
EntityId: LC08_L1TP_043035_20160519_20170223_01_T1
EntityId: LC08_L1TP_042035_20160512_20170223_01_T1
EntityId: LC08_L1TP_042035_20160528_20170223_01_T1
EntityId: LC08_L1TP_042036_20160528_20170223_01_T1
EntityId: LC08_L1TP_043035_20160722_20170221_01_T1
EntityId: LC08_L1TP_043035_20160706_20170221_01_T1
EntityId: LC08_L1TP_040037_20160615_20170220_01_T1
EntityId: LC08_L1TP_041037_20160708_20170222_01_T1
EntityId: LC08_L1TP_040037_20160701_20170222_01_T1
EntityId: LC08_L1TP_041037_20160622_20170222_01_T1
EntityId: LC08_L1TP_041037_20160724_20170222_01_T1
EntityId: LC08_L1TP_040037_20160717_20170222_01_T1
EntityId: LC08_L1TP_041037_20160910_20170222_01_T1
EntityId: LC08_L1TP_040037_20160919_20170222_01_T1
EntityId: LC08_L1TP_041037_20160926_20170220_01_T1
EntityId: LC08_L1TP_040037_2016

EntityId: LC08_L1TP_041035_20161129_20170219_01_T1
EntityId: LC08_L1TP_042035_20161206_20170219_01_T1
EntityId: LC08_L1TP_041035_20161028_20170219_01_T1
EntityId: LC08_L1TP_042035_20161019_20170219_01_T1
EntityId: LC08_L1TP_042034_20160731_20180201_01_T1
EntityId: LC08_L1TP_041035_20161129_20170219_01_T1
EntityId: LC08_L1TP_042035_20161206_20170219_01_T1
EntityId: LC08_L1TP_043034_20161026_20170219_01_T1
EntityId: LC08_L1TP_042034_20161019_20170219_01_T1
EntityId: LC08_L1TP_043034_20160908_20170222_01_T1
EntityId: LC08_L1TP_042034_20160901_20170222_01_T1
EntityId: LC08_L1TP_043033_20160823_20170223_01_T1
EntityId: LC08_L1TP_043033_20160417_20170223_01_T1
EntityId: LC08_L1TP_044032_20160408_20170223_01_T1
EntityId: LC08_L1TP_043033_20160924_20170222_01_T1
EntityId: LC08_L1TP_044033_20160915_20170222_01_T1
EntityId: LC08_L1TP_043033_20161026_20170219_01_T1
EntityId: LC08_L1TP_041036_20160606_20170223_01_T1
EntityId: LC08_L1TP_041036_20160708_20170222_01_T1
EntityId: LC08_L1TP_040036_2016

EntityId: LC08_L1TP_045031_20171011_20171024_01_T1
EntityId: LC08_L1TP_044032_20171004_20171014_01_T1
EntityId: LC08_L1TP_043034_20170927_20171013_01_T1
EntityId: LC08_L1TP_044034_20171004_20171014_01_T1
EntityId: LC08_L1TP_043034_20170911_20170927_01_T1
EntityId: LC08_L1TP_044034_20170918_20170929_01_T1
EntityId: LC08_L1TP_043034_20170725_20170810_01_T1
EntityId: LC08_L1TP_044034_20170716_20170727_01_T1
EntityId: LC08_L1TP_044034_20170716_20170727_01_T1
EntityId: LC08_L1TP_042036_20171006_20171023_01_T1
EntityId: LC08_L1TP_042036_20170819_20170826_01_T1
EntityId: LC08_L1TP_042036_20170702_20170715_01_T1
EntityId: LC08_L1TP_042036_20170702_20170715_01_T1
EntityId: LC08_L1TP_043036_20171013_20171024_01_T1
EntityId: LC08_L1TP_042036_20171006_20171023_01_T1
EntityId: LC08_L1TP_042036_20171209_20171223_01_T1
EntityId: LC08_L1TP_042036_20170803_20170812_01_T1
EntityId: LC08_L1TP_042036_20170819_20170826_01_T1
EntityId: LC08_L1TP_042036_20171209_20171223_01_T1
EntityId: LC08_L1TP_042036_2017

EntityId: LC08_L1TP_041036_20170711_20170726_01_T1
EntityId: LC08_L1TP_041036_20170727_20170810_01_T1
EntityId: LC08_L1TP_041036_20170727_20170810_01_T1
EntityId: LC08_L1TP_040036_20170805_20170812_01_T1
EntityId: LC08_L1TP_042036_20170803_20170812_01_T1
EntityId: LC08_L1TP_041036_20170828_20170914_01_T1
EntityId: LC08_L1TP_041036_20171218_20171224_01_T1
EntityId: LC08_L1TP_041036_20171218_20171224_01_T1
EntityId: LC08_L1TP_041035_20170524_20170614_01_T1
EntityId: LC08_L1TP_041036_20170524_20170614_01_T1
EntityId: LC08_L1TP_042035_20170531_20170615_01_T1
EntityId: LC08_L1TP_041035_20170524_20170614_01_T1
EntityId: LC08_L1TP_041035_20170711_20170726_01_T1
EntityId: LC08_L1TP_042035_20170718_20170728_01_T1
EntityId: LC08_L1TP_042036_20170718_20170728_01_T1
EntityId: LC08_L1TP_041036_20171031_20171109_01_T1
EntityId: LC08_L1TP_041036_20170828_20170914_01_T1
EntityId: LC08_L1TP_042036_20170904_20170916_01_T1
EntityId: LC08_L1TP_041035_20170929_20171013_01_T1
EntityId: LC08_L1TP_041035_2017